In [ ]:
#EDA

In [1]:
%run -i "path.ipynb"
path=ipath+"\KPMG_VI_New_raw_data_update_final.xlsx"
import warnings
import pandas as pd
warnings.filterwarnings('ignore')
tran=pd.read_excel(path, sheet_name="Transactions", header=1)
newcust=pd.read_excel(path, sheet_name="NewCustomerList", header=1)
cdemo=pd.read_excel(path, sheet_name="CustomerDemographic", header=1)
cadd=pd.read_excel(path, sheet_name="CustomerAddress", header=1)

In [2]:
print("Transactions ",tran.shape)
print("CustomerDemographic ",cdemo.shape)
print("CustomerAddress ",cadd.shape)

Transactions  (20000, 13)
CustomerDemographic  (4000, 13)
CustomerAddress  (3999, 6)


In [ ]:
#CustomerDemographic dataset cleaning

In [3]:
cdemo.head()

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,default,owns_car,tenure
0,1,Laraine,Medendorp,F,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,"""'",Yes,11.0
1,2,Eli,Bockman,Male,81,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,<script>alert('hi')</script>,Yes,16.0
2,3,Arlin,Dearle,Male,61,1954-01-20,Recruiting Manager,Property,Mass Customer,N,2018-02-01 00:00:00,Yes,15.0
3,4,Talbot,NaN,Male,33,1961-10-03,NaN,IT,Mass Customer,N,() { _; } >_[$($())] { touch /tmp/blns.shellsh...,No,7.0
4,5,Sheila-kathryn,Calton,Female,56,1977-05-13,Senior Editor,NaN,Affluent Customer,N,NIL,Yes,8.0


In [4]:
cdemo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 13 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   customer_id                          4000 non-null   int64         
 1   first_name                           4000 non-null   object        
 2   last_name                            3875 non-null   object        
 3   gender                               4000 non-null   object        
 4   past_3_years_bike_related_purchases  4000 non-null   int64         
 5   DOB                                  3913 non-null   datetime64[ns]
 6   job_title                            3494 non-null   object        
 7   job_industry_category                3344 non-null   object        
 8   wealth_segment                       4000 non-null   object        
 9   deceased_indicator                   4000 non-null   object        
 10  default     

In [5]:
cdemo.duplicated().sum()

0

In [ ]:
#there are no duplicates in this dataset

In [6]:
#drop default column as it contains irrelevant values
cdemo=cdemo.drop(['default'], axis=1)

In [7]:
#check null values in each column
round((cdemo.isnull().sum())).sort_values(ascending=False)

job_industry_category                  656
job_title                              506
last_name                              125
DOB                                     87
tenure                                  87
customer_id                              0
first_name                               0
gender                                   0
past_3_years_bike_related_purchases      0
wealth_segment                           0
deceased_indicator                       0
owns_car                                 0
dtype: int64

In [8]:
#for object columns impute values with n/a
cdemo['job_industry_category']=cdemo['job_industry_category'].fillna("na")
cdemo['job_title']=cdemo['job_title'].fillna("na")
cdemo['last_name']=cdemo['last_name'].fillna("na")

In [9]:
cdemo['deceased_indicator'].value_counts()

N    3998
Y       2
Name: deceased_indicator, dtype: int64

In [10]:
#removing the customer ids for deceased customers
cdemo.drop(cdemo[cdemo['deceased_indicator']=="Y"].index, inplace=True, axis=0)
cdemo.shape

(3998, 12)

In [ ]:
#as the dataset is for the timeperiod of 2017, age will be calculated as of 2017

In [11]:
cdemo['DOB']=pd.to_datetime(cdemo['DOB'])
current_year=2017
cdemo['Age']=current_year-cdemo['DOB'].dt.year

In [12]:
#impute age and tenure with mode
agemode=int(cdemo['Age'].mode())
tenuremode=int(cdemo['tenure'].mode())
cdemo['Age'].fillna(agemode, inplace=True)
cdemo['tenure'].fillna(tenuremode, inplace=True)
cdemo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3998 entries, 0 to 3999
Data columns (total 13 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   customer_id                          3998 non-null   int64         
 1   first_name                           3998 non-null   object        
 2   last_name                            3998 non-null   object        
 3   gender                               3998 non-null   object        
 4   past_3_years_bike_related_purchases  3998 non-null   int64         
 5   DOB                                  3911 non-null   datetime64[ns]
 6   job_title                            3998 non-null   object        
 7   job_industry_category                3998 non-null   object        
 8   wealth_segment                       3998 non-null   object        
 9   deceased_indicator                   3998 non-null   object        
 10  owns_car    

In [13]:
print("Min Age", cdemo['Age'].min())
print("Max Age", cdemo['Age'].max())

Min Age 15.0
Max Age 174.0


In [14]:
cdemo[cdemo['Age']>90]

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,Age
33,34,Jephthah,Bachmann,U,59,1843-12-21,Legal Assistant,IT,Affluent Customer,N,No,20.0,174.0


In [15]:
#age 174 is data error, so we remove this from dataset
#removing the customer ids for deceased customers
cdemo.drop(cdemo[cdemo['Age']>99].index, inplace=True, axis=0)
print("Min Age", cdemo['Age'].min())
print("Max Age", cdemo['Age'].max())

Min Age 15.0
Max Age 86.0


In [16]:
cdemo['Age'].value_counts()

39.0    304
40.0    204
43.0    152
41.0    152
37.0    123
44.0    122
38.0    118
42.0    114
31.0    114
58.0     93
32.0     91
22.0     90
46.0     88
53.0     85
19.0     80
23.0     79
36.0     79
48.0     78
55.0     76
20.0     74
28.0     74
25.0     74
50.0     73
30.0     72
29.0     72
60.0     71
52.0     69
49.0     69
21.0     67
51.0     65
61.0     63
63.0     61
56.0     60
18.0     60
24.0     60
54.0     59
57.0     58
62.0     58
47.0     56
59.0     55
45.0     54
27.0     53
26.0     48
33.0     43
17.0     42
35.0     41
34.0     40
16.0     34
64.0     19
15.0      6
73.0      1
86.0      1
82.0      1
74.0      1
77.0      1
Name: Age, dtype: int64

In [17]:
#creating age groups for better analysis
bins=[13,20,27,34,41,51,65,99]
labels=["13-19","20-26","27-33","34-40","41-50","51-65","66+"]
cdemo["Age_Group"]=pd.cut(cdemo["Age"], bins=bins, labels=labels)
cdemo["Age_Group"].value_counts()

34-40    1021
41-50     871
51-65     827
27-33     506
20-26     471
13-19     296
66+         5
Name: Age_Group, dtype: int64

In [18]:
#as we have age and age groups columns, DOB column can be dropped
cdemo=cdemo.drop('DOB', axis=1)
cdemo.head()

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,Age,Age_Group
0,1,Laraine,Medendorp,F,93,Executive Secretary,Health,Mass Customer,N,Yes,11.0,64.0,51-65
1,2,Eli,Bockman,Male,81,Administrative Officer,Financial Services,Mass Customer,N,Yes,16.0,37.0,34-40
2,3,Arlin,Dearle,Male,61,Recruiting Manager,Property,Mass Customer,N,Yes,15.0,63.0,51-65
3,4,Talbot,na,Male,33,na,IT,Mass Customer,N,No,7.0,56.0,51-65
4,5,Sheila-kathryn,Calton,Female,56,Senior Editor,na,Affluent Customer,N,Yes,8.0,40.0,34-40


In [19]:
#check null values in each column
round((cdemo.isnull().sum())).sort_values(ascending=False)

customer_id                            0
first_name                             0
last_name                              0
gender                                 0
past_3_years_bike_related_purchases    0
job_title                              0
job_industry_category                  0
wealth_segment                         0
deceased_indicator                     0
owns_car                               0
tenure                                 0
Age                                    0
Age_Group                              0
dtype: int64

In [20]:
#normalize gender column
cdemo['gender'].value_counts()

Female    2036
Male      1871
U           87
F            1
Femal        1
M            1
Name: gender, dtype: int64

In [21]:
cdemo['gender']=cdemo['gender'].replace('Female', 'F').replace('Femal', 'F').replace('Male','M')
cdemo['gender'].value_counts()

F    2038
M    1872
U      87
Name: gender, dtype: int64

In [ ]:
#CustomerAddress dataset cleaning

In [22]:
cadd.head()

,customer_id,address,postcode,state,country,property_valuation
0,1,060 Morning Avenue,2016,New South Wales,Australia,10
1,2,6 Meadow Vale Court,2153,New South Wales,Australia,10
2,4,0 Holy Cross Court,4211,QLD,Australia,9
3,5,17979 Del Mar Point,2448,New South Wales,Australia,4
4,6,9 Oakridge Court,3216,VIC,Australia,9


In [23]:
#check null values in each column
round((cadd.isnull().sum())).sort_values(ascending=False)

customer_id           0
address               0
postcode              0
state                 0
country               0
property_valuation    0
dtype: int64

In [24]:
cadd.duplicated().sum()

0

In [25]:
cadd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3999 entries, 0 to 3998
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   customer_id         3999 non-null   int64 
 1   address             3999 non-null   object
 2   postcode            3999 non-null   int64 
 3   state               3999 non-null   object
 4   country             3999 non-null   object
 5   property_valuation  3999 non-null   int64 
dtypes: int64(3), object(3)
memory usage: 187.6+ KB


In [26]:
cadd['state'].value_counts()

NSW                2054
VIC                 939
QLD                 838
New South Wales      86
Victoria             82
Name: state, dtype: int64

In [27]:
#normalizing values in state column
cadd['state']=cadd['state'].replace('New South Wales', 'NSW').replace('Victoria', 'VIC')
cadd['state'].value_counts()

NSW    2140
VIC    1021
QLD     838
Name: state, dtype: int64

In [ ]:
#Transactions dataset cleaning

In [28]:
tran.head()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
0,1,2,2950,2017-02-25,0.0,Approved,Solex,Standard,medium,medium,71.49,53.62,41245.0
1,2,3,3120,2017-05-21,1.0,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,41701.0
2,3,37,402,2017-10-16,0.0,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,36361.0
3,4,88,3135,2017-08-31,0.0,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10,36145.0
4,5,78,787,2017-10-01,1.0,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,42226.0


In [29]:
tran.duplicated().sum()

0

In [30]:
tran.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   transaction_id           20000 non-null  int64         
 1   product_id               20000 non-null  int64         
 2   customer_id              20000 non-null  int64         
 3   transaction_date         20000 non-null  datetime64[ns]
 4   online_order             19640 non-null  float64       
 5   order_status             20000 non-null  object        
 6   brand                    19803 non-null  object        
 7   product_line             19803 non-null  object        
 8   product_class            19803 non-null  object        
 9   product_size             19803 non-null  object        
 10  list_price               20000 non-null  float64       
 11  standard_cost            19803 non-null  float64       
 12  product_first_sold_date  19803 n

In [31]:
#converting excel date values to datetime format
tran['product_first_sold_date']=pd.to_datetime(tran['product_first_sold_date'], unit='D', origin='1899-12-30')

In [32]:
#check null values in each column
round((tran.isnull().sum())).sort_values(ascending=False)

online_order               360
brand                      197
product_line               197
product_class              197
product_size               197
standard_cost              197
product_first_sold_date    197
transaction_id               0
product_id                   0
customer_id                  0
transaction_date             0
order_status                 0
list_price                   0
dtype: int64

In [33]:
tran[tran.isnull().any(axis=1)]

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
97,98,49,333,2017-06-23,NaN,Approved,Trek Bicycles,Road,medium,medium,533.51,400.13,2003-07-21
136,137,0,431,2017-09-23,0.0,Approved,NaN,NaN,NaN,NaN,1942.61,NaN,NaT
159,160,0,3300,2017-08-27,0.0,Approved,NaN,NaN,NaN,NaN,1656.86,NaN,NaT
166,167,90,3177,2017-04-26,NaN,Approved,Norco Bicycles,Standard,low,medium,363.01,290.41,2005-05-10
169,170,6,404,2017-10-16,NaN,Approved,OHM Cycles,Standard,high,medium,227.88,136.73,2003-08-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19635,19636,98,1389,2017-07-26,NaN,Approved,Trek Bicycles,Standard,high,medium,358.39,215.03,2004-01-16
19793,19794,0,2860,2017-01-13,0.0,Approved,NaN,NaN,NaN,NaN,868.56,NaN,NaT
19843,19844,3,8,2017-01-28,NaN,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,2003-07-21
19859,19860,0,2468,2017-06-24,1.0,Approved,NaN,NaN,NaN,NaN,1497.43,NaN,NaT


In [34]:
#dropping null values from transaction dataset
tran.dropna(inplace=True)

In [35]:
tran.shape

(19445, 13)

In [36]:
#add profit column
tran['profit']=tran['list_price']-tran['standard_cost']
tran[['list_price', 'standard_cost', 'profit']]

,list_price,standard_cost,profit
0,71.49,53.62,17.87
1,2091.47,388.92,1702.55
2,1793.43,248.82,1544.61
3,1198.46,381.10,817.36
4,1765.30,709.48,1055.82
...,...,...,...
19995,2005.66,1203.40,802.26
19996,416.98,312.74,104.24
19997,1636.90,44.71,1592.19
19998,227.88,136.73,91.15


In [37]:
print("Transactions ",tran.shape)
print("CustomerDemographic ",cdemo.shape)
print("CustomerAddress ",cadd.shape)

Transactions  (19445, 14)
CustomerDemographic  (3997, 13)
CustomerAddress  (3999, 6)


In [38]:
#unique customer ids in all 3 datasets
print("Transactions ",tran['customer_id'].sort_values().unique())
print("CustomerDemographic ",cdemo['customer_id'].sort_values().unique())
print("CustomerAddress ",cadd['customer_id'].sort_values().unique())

Transactions  [   1    2    3 ... 3499 3500 5034]
CustomerDemographic  [   1    2    3 ... 3998 3999 4000]
CustomerAddress  [   1    2    4 ... 4001 4002 4003]


In [ ]:
#CustomerDemographic has upto 4000 ids, whereas CustomerAddress has extra 3 ids, Transaction has 1 extra id

In [44]:
#merge all 3 datasets, assuming CustomerDemographic as master data
merged=cdemo.merge(cadd,on='customer_id').merge(tran, on='customer_id')

In [45]:
merged.shape

(19400, 31)

In [46]:
merged['customer_id'].sort_values().unique()

array([   1,    2,    4, ..., 3498, 3499, 3500], dtype=int64)

In [47]:
#check null values in each column
round((merged.isnull().sum())).sort_values(ascending=False)

customer_id                            0
country                                0
product_first_sold_date                0
standard_cost                          0
list_price                             0
product_size                           0
product_class                          0
product_line                           0
brand                                  0
order_status                           0
online_order                           0
transaction_date                       0
product_id                             0
transaction_id                         0
property_valuation                     0
state                                  0
first_name                             0
postcode                               0
address                                0
Age_Group                              0
Age                                    0
tenure                                 0
owns_car                               0
deceased_indicator                     0
wealth_segment  

In [49]:
#exporting merged dataset for further analysis
%run -i "path.ipynb"
outpath=ipath+"\KPMG_VI_merged.xlsx"
merged.to_excel(outpath, sheet_name='MergedDataset', index=False)